In [ ]:
import telebot

import sqlite3 as sql


bot = telebot.TeleBot('1543309879:AAGZ3exoeK89P_TWYDpzRfDbxkabNhIoSV8')

keyboard1 = telebot.types.ReplyKeyboardMarkup(True, True)
keyboard1.row("Клієнт \U0001f4da", "Викладач \U0001F469\u200D\U0001F3EB")
              
     
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Вкажіть будь ласка ви клієнт чи викладач?', reply_markup=keyboard1)

    
@bot.message_handler(content_types=['text'])
def send_text(message):   
    if message.text == "Клієнт \U0001f4da":
        msg =bot.send_message(message.chat.id,  "Вкажіть піб")
        bot.register_next_step_handler(msg,step_for_client)
            
    elif message.text == "Викладач \U0001F469\u200D\U0001F3EB" :
            msg=bot.send_message(message.chat.id, 'Введіть пароль')  
            bot.register_next_step_handler(msg, step_for_teacher)
    else: 
            bot.send_message(message.chat.id, 'Ви щось не те натикали')
            
def step_for_client(message):
    conn = sql.connect('school5.db')
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS student (student_id text, student_name text,student_phone text,student_shedule text,student_teacher text)")
    id = message.from_user.id
    name=message.text
    cur.execute("SELECT student_name FROM student WHERE student_name = '{}';".format(name))
    all_results = cur.fetchall()
    if len(all_results)== 0:
        cur.execute("INSERT INTO student VALUES('{}', '{}', '{}', '{}', '{}');".format(id, name,' ',' ',' '))  
        msg=bot.send_message(message.chat.id, 'Введіть номер телефону у форматі +380.........')
  
        bot.register_next_step_handler(msg, step_for_client_phone)
    else:
        bot.send_message(message.chat.id, 'Ви вже записані на пробне.Ось ваші данні. Хочете щось змінити?')
    conn.commit()

    
def step_for_client_phone(message): 
    
    conn = sql.connect('school5.db')
    cur = conn.cursor()
    id = message.from_user.id
    phone=message.text  
    cur.execute(f"UPDATE student SET student_phone='{phone}' WHERE  student_id='{id}'")
    #cur.execute("UPDATE student SET student_phone='{}' WHERE  student_id='{}';".format(phone,id))  
def step_for_teacher(message):   
     if message.text != "1111":
        msg = bot.send_message(message.chat.id, 'Пароль не вірний. Спробуйте ще раз')
        bot.register_next_step_handler(msg, step_for_teacher)
        return
     else:
        bot.send_message(message.chat.id, 'Ви увійшли в систему')
bot.polling()
